# Using MLRUN with Dask Distributed Jobs

In [ ]:
# recommended, installing the exact package versions
#!pip install dask==2.9.1 distributed==2.9.1 kubernetes==10.0.0 dask_kubernetes==0.10.0 kubernetes-asyncio==10.0.0 msgpack==0.6.2

## Writing a function code

In [1]:
# function that will be distributed 
def inc(x):
    return x+2

The MLRun context in the case of Dask will have an extra param `dask_client`
which is initialized based on the function spec (below), and can be used to submit Dask commands.

In [2]:
# wrapper function, uses the dask client object
def hndlr(context, x=1,y=2):
    context.logger.info('params: x={},y={}'.format(x,y))
    print('params: x={},y={}'.format(x,y))
    x = context.dask_client.submit(inc, x)
    print(x)
    print(x.result())
    context.log_result('y', x.result())

In [3]:
# nuclio: end-code
# marks the end of a code section, do not delete

In [4]:
# nuclio: ignore
import nuclio

In [5]:
from mlrun import new_function, mlconf, code_to_function, mount_v3io, NewTask
mlconf.dbpath = 'http://mlrun-api:8080'

## Define a Dask function object
dask functions can be local (local workers), or remote (use containers in the cluster),
in the case of `remote` users can specify the number of replica (optional) or leave blank for auto-scale.

We use `code_to_function()` which packs the function code into the function object/yaml (eliminate the need to update the function image), we can use `new_function()` if the function code is part of the image or can be remotely mounted (e.g. via v3io mount).

Dask function spec have several unique attributes:
* **.remote** - bool, use local or clustered dask
* **.replicas** - number of desired replicas, keep 0 for auto-scale
* **.min_replicas, .max_replicas** - set replicas range for auto-scale
* **.scheduler_timeout** - cluster will be killed after timeout (inactivity), default is `'60 minutes'`
* **.nthreads** - number of worker threads
* **.kfp_image** - optional, container image to use by KFP Pipeline runner (default to mlrun/dask)

If you want to access the dask dashboard from remote you need to use `NodePort` service type (set **.service_type** to 'NodePort'), and the external IP need to be specified in mlrun configuration (`mlconf.remote_host`), this will be set automatically if you are running on an Iguazio cluster.
If you want to use the `NodePort` for remote access to the scheduler you need to also specify `function.use_remote=True`.

In [6]:
# create the function from the notebook code + annotations, add volumes
dsf = code_to_function('mydask', runtime='dask').apply(mount_v3io())

In [7]:
dsf.spec.image = 'daskdev/dask:2.9.1'
dsf.spec.remote = True
dsf.spec.replicas = 1
dsf.spec.service_type = 'NodePort'
dsf.spec.image_pull_policy = 'Always'

## Build the function with extra packages
We can skip the build section if we dont add packages (instead need to specify the image e.g. `dsf.spec.image='daskdev/dask:2.9.1'`) 

In [8]:
# uncomment if you want to add packages to the workers and build a new image
# dsf.build_config(base_image='daskdev/dask:2.9.1', commands=['pip install pandas'])
# dsf.deploy()

## Run a task using our distributed dask function (cluster)

In [9]:
myrun = dsf.run(handler=hndlr, params={'x': 12})

[mlrun] 2020-02-22 00:54:50,857 starting run mydask-hndlr uid=054dad9cf1084a399dafef3e917c34bb  -> http://mlrun-api:8080
[mlrun] 2020-02-22 00:54:51,339 trying dask client at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786
[mlrun] 2020-02-22 00:54:51,349 using remote dask scheduler (mlrun-mydask-d3b406f6-c) at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786


[mlrun] 2020-02-22 00:54:51,359 params: x=12,y=2
params: x=12,y=2
<Future: pending, key: inc-4902ea8dcd9e6250fc5e9a2c2f2b9e3d>
14

[mlrun] 2020-02-22 00:54:51,397 run ended with state 


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...7c34bb,0,Feb 22 00:54:50,completed,mydask-hndlr,kind=daskowner=adminhost=jupyter-68bdf65845-6rc8r,,x=12,y=14,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 054dad9cf1084a399dafef3e917c34bb  , !mlrun logs 054dad9cf1084a399dafef3e917c34bb 
[mlrun] 2020-02-22 00:54:51,514 run executed, status=completed


In [10]:
# get the function status and addresses
dsf.status.to_dict()

{'scheduler_address': 'tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786',
 'cluster_name': 'mlrun-mydask-d3b406f6-c',
 'node_ports': {'dashboard': 30731, 'scheduler': 30399}}

### Accesing the dask client directly
You can get the dask client object and cluster information by reading the client object.

> Note: the cluster can timeout, when you call the client MLrun will also verify the cluster is live and active and if not it will restart the dask cluster and refresh the function object with the latest addresses and status.

In [11]:
c = dsf.client

[mlrun] 2020-02-22 00:54:54,780 trying dask client at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786
[mlrun] 2020-02-22 00:54:54,788 using remote dask scheduler (mlrun-mydask-d3b406f6-c) at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786


### Access a dask function using the DB
If we want to access the dask function (or its cluster), we can load the function object from the DB (assuming we already .run() or .save() it).

This can be useful if we want to load the same function in a different notebook or container, or if we restarted our notebook

In [12]:
from mlrun import import_function
# Functions url: db://<project>/<name>[:tag]
dsf_obj = import_function('db://default/mydask')
c = dsf_obj.client

[mlrun] 2020-02-22 00:55:01,527 trying dask client at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786
[mlrun] 2020-02-22 00:55:01,537 using remote dask scheduler (mlrun-mydask-d3b406f6-c) at: tcp://mlrun-mydask-d3b406f6-c.default-tenant:8786


## Building a Pipeline using dask functions

In [16]:
import kfp
from kfp import dsl
from mlrun import run_pipeline

In [17]:
@dsl.pipeline(name="dask_pipeline")
def dask_pipe(x=1,y=10):
    # use_db option will use a function (DB) pointer instead of adding the function spec to the YAML
    myrun = dsf.as_step(NewTask(handler=hndlr, name="dask_pipeline", params={'x': x, 'y': y}), use_db=True)
    # is the step (dask client) need v3io access u should add: .apply(mount_v3io())
    myrun.container.set_image_pull_policy('Always')

In [18]:
# for pipeline debug
kfp.compiler.Compiler().compile(dask_pipe, 'daskpipe.yaml', type_check=False)

[mlrun] 2020-02-22 00:57:06,512 saving function: mydask, tag: latest


/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "10".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [19]:
arguments={'x':4,'y':-5}
artifacts_path = '/User/test'
run_pipeline(dask_pipe, arguments, artifacts_path=artifacts_path,
             run="DaskExamplePipeline", experiment="dask pipe")

[mlrun] 2020-02-22 00:57:17,427 saving function: mydask, tag: latest


[mlrun] 2020-02-22 00:57:17,485 Pipeline run id=b8ee6559-7a98-4093-bddb-daac83c6a865, check UI or DB for progress


'b8ee6559-7a98-4093-bddb-daac83c6a865'